In [6]:
import pandas as pd


In [7]:
df=pd.read_csv('spamhamdata.csv',sep='\t',names=["label","mail"])
df.head()

,label,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

        


In [19]:
def preprocessText(mail):
    mail=re.sub('[^a-zA-Z]',' ',mail)
    mail=mail.lower()
    mail=mail.split()
    newmail_list=[ lemmatizer.lemmatize(word) for word in mail if not word in stopwords.words('english')]
    newmail=' '.join(newmail_list)
    # print(newmail)
    return newmail

In [16]:
from tqdm import tqdm

In [20]:
cleaned_mails=[]
for mail in  tqdm(df['mail']):
    cleaned_mails.append(preprocessText(mail))    


100%|██████████████████████████████████████████████████████████████████████████████| 5572/5572 [02:05<00:00, 44.52it/s]


In [28]:
# cleaned_mails

In [22]:
df['cleaned_mail']=cleaned_mails

In [23]:
df

,label,mail,cleaned_mail
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,nd time tried contact u u pound prize claim ea...
5568,ham,Will ü b going to esplanade fr home?,b going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood suggestion
5570,ham,The guy did some bitching but I acted like i'd...,guy bitching acted like interested buying some...


In [25]:
df.to_csv('cleaned_mails.csv',index=False)

In [75]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [76]:
cv=CountVectorizer(max_features=2500)
tf=TfidfVectorizer(max_features=2500)

In [59]:
x=cv.fit_transform(df['cleaned_mail']).toarray()

In [60]:
x.shape

(5572, 2500)

In [61]:
cv.get_feature_names_out()

array(['aah', 'aathi', 'abi', ..., 'yuo', 'yup', 'zed'], dtype=object)

In [62]:
y=df['label']

In [63]:
y

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: label, Length: 5572, dtype: object

In [64]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [65]:
x_train.shape,y_train.shape

((4457, 2500), (4457,))

In [66]:
x_test.shape,y_test.shape

((1115, 2500), (1115,))

In [67]:
from sklearn.naive_bayes import MultinomialNB

In [68]:
model=MultinomialNB()

In [69]:
model.fit(x_train,y_train)

MultinomialNB()

In [70]:
y_pred=model.predict(x_test)

In [71]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [72]:
accuracy_score(y_test,y_pred)

0.9847533632286996

In [73]:
confusion_matrix(y_test,y_pred)

array([[955,  11],
       [  6, 143]], dtype=int64)

In [74]:

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       966
        spam       0.93      0.96      0.94       149

    accuracy                           0.98      1115
   macro avg       0.96      0.97      0.97      1115
weighted avg       0.99      0.98      0.98      1115

